<div style="text-align: center; background-color: #5A96E3; font-family: 'Trebuchet MS', Arial, sans-serif; color: white; padding: 20px; font-size: 40px; font-weight: bold; border-radius: 0 0 0 0; box-shadow: 0px 6px 8px rgba(0, 0, 0, 0.2);">
  Model</div>

## Import libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import confusion_matrix
from sklearn.metrics import mean_squared_error

## Read csv file

In [2]:
df = pd.read_csv('data_footballer_processed.csv', parse_dates=['Birth Date'])
df.head(10)

,Name,Height,Weight,Preferred Foot,Birth Date,Age,Nation,Club,League,Preferred Positions,...,POT,Value,Wage,Ball Skills,Defence,Mental,Passing,Physical,Shooting,Goalkeeper
0,Erling Haaland,195,94,Left,2000-07-21,23,Norway,Manchester City,England Premier League (1),ST,...,94,157000000.0,340000.0,80.5,38.0,80.2,59.0,83.7,84.0,10.4
1,Kylian Mbappé,182,75,Right,1998-12-20,24,France,Paris Saint-Germain,France Ligue 1 (1),"ST, LW",...,94,153500000.0,225000.0,92.5,33.0,76.7,78.3,89.0,82.2,8.4
2,Kevin De Bruyne,181,75,Right,1991-06-28,32,Belgium,Manchester City,England Premier League (1),"CM, CAM",...,91,103000000.0,350000.0,89.0,61.5,84.0,94.3,75.7,83.1,11.2
3,Harry Kane,188,85,Right,1993-07-28,30,England,Bayern München,Germany 1. Bundesliga (1),ST,...,90,119500000.0,170000.0,84.5,42.0,81.3,85.0,75.9,86.5,10.8
4,Thibaut Courtois,199,96,Left,1992-05-11,31,Belgium,Real Madrid,Spain Primera Division (1),GK,...,90,63000000.0,250000.0,18.0,17.0,41.5,27.3,54.0,22.4,86.6
5,Robert Lewandowski,185,81,Right,1988-08-21,35,Poland,FC Barcelona,Spain Primera Division (1),ST,...,90,58000000.0,340000.0,88.0,30.5,81.0,76.7,81.1,87.8,10.2
6,Karim Benzema,185,81,Right,1987-12-19,35,France,Al Ittihad,Saudi Pro League (1),"CF, ST",...,90,51000000.0,95000.0,89.0,21.0,77.7,80.0,79.3,84.6,8.2
7,Lionel Messi,169,67,Left,1987-06-24,36,Argentina,Inter Miami,USA Major League Soccer (1),"CF, CAM",...,90,41000000.0,23000.0,94.5,29.5,75.2,88.0,79.4,83.6,10.8
8,Rúben Dias,187,82,Right,1997-05-14,26,Portugal,Manchester City,England Premier League (1),CB,...,90,97500000.0,250000.0,69.5,89.0,73.2,71.0,70.1,48.4,9.4
9,Vini Jr.,176,73,Right,2000-07-12,23,Brazil,Real Madrid,Spain Primera Division (1),LW,...,94,121500000.0,310000.0,91.0,21.5,69.5,78.3,84.3,71.4,7.2


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10020 entries, 0 to 10019
Data columns (total 21 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Name                 10020 non-null  object        
 1   Height               10020 non-null  int64         
 2   Weight               10020 non-null  int64         
 3   Preferred Foot       10020 non-null  object        
 4   Birth Date           10020 non-null  datetime64[ns]
 5   Age                  10020 non-null  int64         
 6   Nation               10020 non-null  object        
 7   Club                 10020 non-null  object        
 8   League               10020 non-null  object        
 9   Preferred Positions  10020 non-null  object        
 10  OVR                  10020 non-null  int64         
 11  POT                  10020 non-null  int64         
 12  Value                10020 non-null  float64       
 13  Wage                 10020 non-

## Model

- **We create a model to predict Value of a footballer**, there are features in dataset that influence the target variable `Value`:
  - Height
  - Weight
  - Age
  - OVR
  - POT
  - Ball Skill
  - Defence
  - Mental
  - Passing
  - Physical
  - Shooting
  - Goalkeeper
- Predicting the value of a footballer is a regression task where the goal is to estimate a continuous variable (e.g., the monetary value of the player). Different regression algorithms can be considered for this task, including **Decision Tree Regression**, **Linear Regression**, and **Random Forest Regression**.
- Here's a detailed explanation of why you might choose each of these algorithms:
  - **Linear Regression**: Linear Regression is a simple and interpretable algorithm that assumes a linear relationship between the features and the target variable. It's suitable when the relationship between the features and the target is approximately linear.
  - **Decision Tree Regression**: Decision trees are versatile and can capture non-linear relationships in the data. They are useful when the relationship between features and the target is complex and non-linear.
  - **Random Forest Regression**: Random Forest is an ensemble method that builds multiple decision trees and averages their predictions. It is robust, handles complex relationships well, and is less prone to overfitting than a single decision tree.


### Preprocessing

- Categorical variables, such as `League`, `Nation`, and `Preferred Positions`, can significantly influence the target variable `Value` in a dataset.
- In the context of a machine learning model, these categorical features often hold valuable information about the characteristics of entities like players in a sports dataset.
- To incorporate these categorical variables into a machine learning model, it's essential to encode them appropriately.
- One common method is one-hot encoding, which transforms categorical variables into a binary format, creating binary columns for each category. This encoding preserves the distinctiveness of categories while allowing the model to effectively use them as features for predicting the target variable.

In [4]:
# encode columns league and nation
one_hot_nation_league_df = pd.get_dummies(df[['League', 'Nation']])
# encode column position
position_df = df['Preferred Positions']
one_hot_position_df = position_df.str.get_dummies(', ')
# custom order from GK -> ST
custom_order = ['GK', 'LWB', 'LB', 'CB', 'RB', 'RWB', 'CDM', 'CM', 'LM', 'RM', 'CAM', 'LW', 'RW', 'CF', 'ST']
one_hot_position_df = one_hot_position_df[custom_order]
# merge encode df
one_hot_df = one_hot_nation_league_df.merge(one_hot_position_df, left_index=True, right_index=True)
one_hot_df

,League_Argentina Primera Division (1),League_Australia A-League (1),League_Austria Bundesliga (1),League_Belgium Pro League (1),League_CONMEBOL Libertadores,League_CONMEBOL Sudamericana,League_China Super League (1),League_Denmark Superliga (1),League_England Championship (2),League_England League One (3),...,RWB,CDM,CM,LM,RM,CAM,LW,RW,CF,ST
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,1,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10015,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10016,0,0,0,0,0,0,0,0,1,0,...,1,0,0,0,0,0,0,0,0,0
10017,0,0,0,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
10018,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


- Merge one_hot with numeric columns in df

In [5]:
numeric_columns = ['Height', 'Weight', 'Age', 'OVR', 'POT', 'Ball Skills', 'Defence', 'Mental', 'Passing', 'Physical', 'Shooting', 'Goalkeeper']
numeric_df = df[numeric_columns]
encoded_df = numeric_df.merge(one_hot_df, left_index=True, right_index=True)
encoded_df

,Height,Weight,Age,OVR,POT,Ball Skills,Defence,Mental,Passing,Physical,...,RWB,CDM,CM,LM,RM,CAM,LW,RW,CF,ST
0,195,94,23,91,94,80.5,38.0,80.2,59.0,83.7,...,0,0,0,0,0,0,0,0,0,1
1,182,75,24,91,94,92.5,33.0,76.7,78.3,89.0,...,0,0,0,0,0,0,1,0,0,1
2,181,75,32,91,91,89.0,61.5,84.0,94.3,75.7,...,0,0,1,0,0,1,0,0,0,0
3,188,85,30,90,90,84.5,42.0,81.3,85.0,75.9,...,0,0,0,0,0,0,0,0,0,1
4,199,96,31,90,90,18.0,17.0,41.5,27.3,54.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10015,191,87,21,65,75,16.5,12.5,30.0,31.3,40.4,...,0,0,0,0,0,0,0,0,0,0
10016,177,68,20,65,81,66.0,58.0,58.7,60.7,70.7,...,1,0,0,0,0,0,0,0,0,0
10017,170,60,23,65,74,65.0,58.5,60.3,59.0,58.4,...,0,0,1,0,0,0,0,0,0,0
10018,187,77,25,65,72,48.0,67.5,55.2,52.0,63.1,...,0,0,0,0,0,0,0,0,0,0


## Data Modelling

### Decision Tree Regression

####  Training model

- Training a Decision Tree Regression model on the `X_train` and `y_train` 

In [6]:
# Define X, y
X = encoded_df
y = df['Value']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

- Traning model

In [7]:
my_tree = DecisionTreeRegressor()
my_tree.fit(X_train, y_train)

DecisionTreeRegressor()

- Predict `X_test`, store in `y_pred`

In [8]:
y_pred = my_tree.predict(X_test)

- Excute mean square error

In [9]:
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')

Mean Squared Error: 4509047180638.723


## Model Evaluation

## Model Tuning with Hyperparameters